In [5]:
import h5py
import meshoid
%pylab

F = h5py.File("snapshot_010.hdf5")
rho = np.array(F["PartType0"]["Density"])
zz = np.array(F["PartType0"]["Metallicity"])[:,0]
    
M = meshoid.FromSnapshot("snapshot_010.hdf5",ptype=0)

    # /* Krumholz & Gnedin fitting function for f_H2 as a function of local properties */
    # double tau_fmol = evaluate_NH_from_GradRho(P[i].GradRho,PPP[i].Hsml,SphP[i].Density,PPP[i].NumNgb,1);
    # tau_fmol *= (0.1 + P[i].Metallicity[0]/All.SolarAbundances[0]);
    # if(tau_fmol>0) {
    #     tau_fmol *= 434.78 * All.UnitDensity_in_cgs * All.UnitLength_in_cm * All.HubbleParam;
    #     y = 0.756 * (1 + 3.1*pow(P[i].Metallicity[0]/All.SolarAbundances[0],0.365));
    #     y = log(1 + 0.6*y + 0.01*y*y) / (0.6*tau_fmol);
    #     y = 1 - 0.75*y/(1 + 0.25*y);
    #     if(y<0) y=0; if(y>1) y=1;
    #     rateOfSF *= y;
    # } // if(tau_fmol>0)

gradrho = np.sum(M.D(rho)**2,axis=1)**0.5 # magnitude of the density gradient
sigma = rho * rho/gradrho + M.m/(np.pi*(M.h/32**(1./3))**2) # column density esimate
tau_fmol = sigma * (0.1 + zz)
tau_fmol *= 434.78 * 2.088 * 0.7
y = 0.756 * (1 + 3.1*(.1+zz)**0.365)
y = np.log(1 + 0.6*y + 0.01*y*y) / (0.6*tau_fmol)
y = 1 - 0.75*y/(1 + 0.25*y) # the molecular fraction
y = np.clip(y,0,1)

plt.hist(np.log10(y[y>0]),100); plt.yscale('log'); plt.show()

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib
